# Upload data to a S3 bucket

See example here:
http://datasciencedirectory.com/how-to-connect-to-aws-s3-buckets-with-python/

In [11]:
from pathlib import Path
import os
from configparser import ConfigParser
import boto3

In [12]:
repodir = Path("../../") 
inputdir = repodir / "data" / "output-report"
os.listdir(inputdir)

['fireveg-db-references.xlsx',
 'fireveg-trait-records-model.xlsx',
 'fireveg-trait-report-model.xlsx']

Read parameters for connection to S3 bucket

In [13]:
def read_s3params(filename,section="firevegbucket"):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db


In [14]:
filename = repodir / 'secrets' / 's3info'
s3params = read_s3params(filename)

Start a connection to S3 using the connection parameters: 

In [18]:
s3=boto3.client('s3', s3params['region'],
                        aws_access_key_id=s3params['key'],
                  aws_secret_access_key=s3params['secret'])

In [ ]:
Example upload of reports to a folder in a bucket:

In [40]:
inputdir = repodir / "data" / "output-report"
targetfile = 'fireveg-trait-report-model.xlsx'
targetfile = 'fireveg-trait-records-model.xlsx'
targetfile = 'fireveg-field-report-model.xlsx'

In [41]:
with open(inputdir / targetfile, "rb") as f:
  s3.upload_fileobj(f, 'fireveg-db', 'output-report/' + targetfile)

Upload all original field forms to a input folder

In [36]:
inputdir = repodir / "data" / "field-form"

for fp in inputdir.glob('*.xls[mx]'):
    targetname=os.path.basename(fp)
    if not targetname.startswith('~'):
        print(targetname)
        with open(fp, "rb") as f:
            s3.upload_fileobj(f, 'fireveg-db', 'input-field-form/' + targetname)


SthnNSWRF_data_bionet2.xlsx
UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton.xlsx
UNSWFireVegResponse_UplandBasalt_AlexThomsen+DK.xlsx
UNSW_VegFireResponse_KNP AlpAsh_firehistupdate.xlsx
RobertsonRF_data_bionet2.xlsx
UNSW_VegFireResponse_RMK_reformat_Sep2021a.xlsx
UNSW_VegFireResponse_AlpineBogs_reformat_Sep2021.xlsx
UNSW_VegFireResponse_KNP AlpAsh.xlsx
Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm
UNSW_VegFireResponse_DataEntry_Yatteyattah all +DK +Milton_revisedfields_Mar2022.xlsx
